In [60]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [93]:
import ray
import modin
print(ray.__version__)
print(modin.__version__)

ray.shutdown()
ray.init(num_cpus = 8)
import modin.pandas as pd

2019-10-06 17:43:40,349	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-10-06 17:43:40,351	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-06_17-43-40_351280_76380/logs.


0.7.3
0.6.1


2019-10-06 17:43:40,466	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:35967 to respond...
2019-10-06 17:43:40,589	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:11948 to respond...
2019-10-06 17:43:40,596	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2019-10-06 17:43:40,622	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-06_17-43-40_351280_76380/logs.
2019-10-06 17:43:40,626	WARNING services.py:1330 -- WARNING: The default object store size of 5.15 GB will use more than 50% of the available memory on this node (6.61 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2019-10-06 17:43:40,628	INFO services.py:1475 -- Starting the Plasma object store with 5.15 GB memory using /tmp.


In [22]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))

#Note it doesn't like the parse datetime here. 
nyc_taxi_fare = pd.read_csv(nyc_taxi_fare_file, 
                            dtype = nyc_taxi_fare_dtype)

CPU times: user 53.4 ms, sys: 17 ms, total: 70.3 ms
Wall time: 1 s


In [23]:
type(nyc_taxi_fare)

modin.pandas.dataframe.DataFrame

In [24]:
nyc_taxi_fare.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24:57.0,15.3,2011-05-28 14:24:57 UTC,-74.003957,40.707345,-73.987255,40.756599,1
1,04:00.0,10.5,2010-06-13 00:04:00 UTC,-73.985308,40.727445,-73.983152,40.756828,1
2,40:23.0,15.7,2012-06-01 21:40:23 UTC,-73.991942,40.763950,-73.994583,40.718426,1
3,56:00.0,5.0,2014-03-03 21:56:00 UTC,-73.982507,40.692082,-73.968980,40.694112,2
4,21:10.0,6.9,2009-05-07 07:21:10 UTC,-73.997176,40.749256,-73.982095,40.736297,4


In [33]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].mean()

CPU times: user 40.2 ms, sys: 5.62 ms, total: 45.8 ms
Wall time: 42 ms


passenger_count
0     8.191667
1    11.405134
2    12.241952
3    11.260214
4    10.229519
5    11.670223
6    10.476476
Name: fare_amount, dtype: float64

In [89]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(count = len, 
                                                              mean = np.mean,
                                                              std = np.std,
                                                              min = np.min,
                                                              median = np.median)

CPU times: user 44.9 ms, sys: 4.94 ms, total: 49.9 ms
Wall time: 46.5 ms


,count,mean,std,min,median
passenger_count,,,,,
0,24.0,8.191667,4.095376,2.5,7.3
1,6973.0,11.405134,9.739049,2.5,8.5
2,1445.0,12.241952,13.548538,2.5,8.5
3,467.0,11.260214,10.085676,2.5,8.5
4,208.0,10.229519,8.600770,3.3,7.7
5,673.0,11.670223,9.499315,2.5,8.5
6,210.0,10.476476,7.071556,2.5,8.5


In [91]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe()

NameError: name 'nyc_taxi_fare' is not defined

In [90]:
del(nyc_taxi_fare)
ray.shutdown()

## MODIN INTERESTING BITS ##